In [239]:
import pandas as pd
from IPython.display import Markdown
import re

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Lyrics Database

In [240]:
lyrics_df_all = pd.read_csv('../raw_data/20250609_17k_lyrics_eng_fr.csv')

In [241]:
lyrics_df_all.drop(columns='Unnamed: 0', inplace=True)
lyrics_df_all.reset_index(drop=True, inplace=True)

In [242]:
lyrics_df_all.isna().sum()

artist                     0
track_title_clean          0
lyrics_clean               3
artist_lower               0
track_title_clean_lower    0
dtype: int64

In [243]:
lyrics_df_all.groupby('artist').count()['lyrics_clean'].sort_values(ascending=False).head(50)

artist
Booba                       90
Marilyn Manson              89
Tim McGraw                  89
Rascal Flatts               89
Kenny Chesney               86
Grateful Dead               85
Shy'm                       85
George Strait               83
Phish                       83
Muse                        80
Christine and the queens    78
Kery James                  78
Def Leppard                 73
Lionel Richie               73
Jenifer                     72
Carla Bruni                 72
Lara Fabian                 71
Rush                        70
Gradur                      69
Lefa                        68
Zazie                       68
Aya Nakamura                68
Weezer                      68
PNL                         68
Slayer                      67
Korn                        67
Norah Jones                 67
Foo Fighters                66
Céline Dion                 66
Hamza                       66
Lynyrd Skynyrd              66
Megadeth                    65
M

## 1.1 Clean lyrics

In [230]:
lyrics_df_all

,artist,track_title_clean,lyrics_clean,artist_lower,track_title_clean_lower
0,ABBA,"Andante, Andante","Take it easy with me, please Touch me gently l...",abba,"andante, andante"
1,ABBA,As Good As New,I'll never know why I had to go Why I had to p...,abba,as good as new
2,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,abba,bang-a-boomerang
3,ABBA,Cassandra,Down in the street they're all singing and sho...,abba,cassandra
4,ABBA,Chiquitita,"Chiquitita, tell me what's wrong You're enchai...",abba,chiquitita
...,...,...,...,...,...
17299,Therapie Taxi,Candide Crush,"Je suis si sage-sage d'habitude, mon chéri Com...",therapie taxi,candide crush
17300,Therapie Taxi,J'en ai marre,Et j'en ai marre de toi Je sais c'est dur mais...,therapie taxi,j'en ai marre
17301,Therapie Taxi,Avec ta zouz,"N'aie pas le seum, fais-moi la bise Ce soir c'...",therapie taxi,avec ta zouz
17302,Minuit,Flash,"Lumière crue, lumière sans, flash! Étrange sen...",minuit,flash


In [251]:
lyrics_df_all.dropna(inplace=True)

In [252]:
import re

def no_backslash(lyrics):
    lyrics = lyrics.replace('\\', '')
    return lyrics


In [253]:
lyrics_df_all['lyrics_clean'] = lyrics_df_all['lyrics_clean'].apply(no_backslash)

In [268]:
list = lyrics_df_all[(lyrics_df_all.artist == 'Booba') & (lyrics_df_all.track_title_clean == 'GP')]['lyrics_clean']

# 2. Download Gemini model

In [ ]:
# Reload dotenv in notebook

from dotenv import load_dotenv
dot_env_loaded = load_dotenv()
dot_env_loaded

True

In [172]:
import os
google_api_key_exists = 'GOOGLE_API_KEY' in os.environ
google_api_key_exists

True

In [173]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What is the capital of France?",
)

Markdown(response.text)

The capital of France is **Paris**.


In [174]:
prompt = '\n\n'.join([doc.page_content for doc in retrieved_docs])
prompt += "\n\n" + query

NameError: name 'retrieved_docs' is not defined

In [ ]:
from langchain import hub

prompt_template = hub.pull("rlm/rag-prompt")

example_messages = prompt_template.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

# 3. Get lyrics by artist

## 3.1 Define function to get lyrics

In [255]:
def get_lyrics(df, column_name, values_to_find):
    # Rename df if required
    filtered_df = df[['track_title_clean', 'lyrics_clean']][df[column_name].isin([values_to_find])]
    return filtered_df.to_dict(orient='records')

## 3.2. Define variables and call function

In [256]:
artist = 'Booba'

In [257]:
# Define variables
df = lyrics_df_all
column_name = 'artist'
values_to_find = artist


get_lyrics(df, column_name, values_to_find)

[{'track_title_clean': 'Rebel',
  'lyrics_clean': '"Herb...herb is a plant! I mean...herbs are good for everything Why...why these people who want to do so much good for everyone, who call themselves governments and this and that... Why them say you must not use the herb? You see...it make you rebel." Tout passe par la force, rien n\'est acquis de droit Le ciel m\'écrit en morse, on dirait qu\'il me voit Tu m\'dis qu\'ces vagues sont imprenables, ai-je vraiment lе choix? Que des emballagеs sans contenu, que des Dua Lipa High-kick, middle-kick, Sawadikap J\'ai confiance en personne, du coup on m\'trahit pas À un moment donné faut prendre les armes, suffit pas d\'y croire À un moment donné faut rendre les armes, c\'est là qu\'s\'écrit l\'histoire Sénégalais chaud comme Haalpulaar C\'que j\'dis c\'est tellement noir, tu crois au canular J\'veux pas leur or, juste leurs têtes me suffira J\'suis craint comme Voldemort, d\'mande à Linda, elle t\'le dira 45 Scientific, l\'escroc s\'appelait G